In [65]:
import psycopg2
import prices

import concurrent.futures

import db
import math

import pandas as pd
import psycopg2.extras as extras 
import yfinance as yf

import matplotlib.pyplot as plt

from configparser import ConfigParser

def getAveragePriceForDate(ticker: str, date):
  response = db.getPriceForDate(ticker, date)
  if response == None or len(response) == 0:
    return -1
  return response[0][1]

def getAveragePriceForWeek(ticker, initial_date, weekNumber):
  current_date = (initial_date + pd.DateOffset(days = (weekNumber + 1) * 7)).date()
  return getAveragePriceForDate(ticker, current_date)

def removeFromPred(week, table_name):
  try:
    # read connection parameters
    params = config()

    connection = psycopg2.connect(**params)
    cursor = connection.cursor()
        
    query = "DELETE FROM " + table_name + " WHERE week = %s"

    cursor.execute(query, (week,))
    connection.commit()

  except (Exception, psycopg2.Error) as error:
    print("Failed to drop a record from the table: ", error)

def selectPosts(table_name):
  try:
    # read connection parameters
    params = db.config()

    connection = psycopg2.connect(**params)
    cursor = connection.cursor()
        
    postgres_insert_query = "SELECT * FROM " + table_name

    cursor.execute(postgres_insert_query)

    connection.commit()
    result = cursor.fetchall()
    return result

  except (Exception, psycopg2.Error) as error:
    print("Failed to select a record from the table: ", error)

  finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()

def makePredictions(sentiment, alpha_values, init_price):
  if init_price == None or math.isnan(init_price):
    return []
  result = []
  for val in alpha_values:
    if val == None or math.isnan(val):
      result.append((0, 0))
      continue
    if sentiment == 1:
      if val > 0:
        if abs(val) > 1:
          result.append((init_price * val + init_price, 1))
        else:
          result.append((init_price * val + init_price, 1))
      else:
        if abs(val) > 1:
          result.append((0, 0))
        else:
          result.append((init_price * val + init_price, 1))
    else:
      if val > 0:
        if abs(val) > 1:
          result.append((0,0))
        else:
          result.append((0 - init_price * val + init_price, 1))
      else:
        if abs(val) > 1:
          result.append((0 - init_price * val + init_price, 1))
        else:
          result.append((0 - init_price * val + init_price, 1))
  return result

def weekPosted(d):
  return ((pd.to_datetime(d) - pd.to_datetime("1-1-2019", dayfirst=True)).days) // 7

def retrievePredictions(ticker):
  table_name = ticker.lower()
  if '-usd' in table_name:
    table_name = "crypt_" + table_name[:-4]
  else:
    table_name = "stock_" + table_name
  posts = selectPosts(table_name)
  if posts == None:
    return -1

  predictions = []
  for week in range(1, 350):
    predictions.append([week, 0., 0., 0])
  for i in range(len(predictions)):
    for d in {"1-1-2019", "2-1-2019", "3-1-2019", "4-1-2019"}:
      if predictions[i][1] == 0 or predictions[i][1] == -1:
        predictions[i][1] = getAveragePriceForWeek(ticker, pd.to_datetime(d, dayfirst=True), predictions[i][0] - 2)

  def worker(post):
    nonlocal predictions
    sent = post[2]
    week = weekPosted(post[3])
    if week < 210:
      return
    alpha = list(db.selectAccuracy(str(post[1]), 'alpha_table'))
    if len(alpha) == 0:
      return
    alpha = list(alpha[0])[1:]
    init_pricee = predictions[week][1]
    pred = makePredictions(sent, alpha, init_pricee)
    for index in range(len(pred)):
      if pred[index][1] == 1:
        if pred == None or pred[index] == None or pred[index][0] == None:
          print('AAA')
          continue
        if math.isnan(pred[index][0]):
          print('BBB')
          continue
        predictions[week + index][2] += pred[index][0]
        predictions[week + index][3] += pred[index][1]

  pool = concurrent.futures.ThreadPoolExecutor(max_workers=100)

  counter = 0
  for post in posts:
    counter += 1
    pool.submit(worker(post))
  
  pool.shutdown(wait=True)
    
  for index in range(len(predictions)):
    if predictions[index][3] and predictions[index][3] != 0:
      predictions[index][2] = predictions[index][2] / predictions[index][3]
  tuple_predictions = []
  for pred in predictions:
    tuple_predictions.append(tuple(pred))
  return tuple_predictions

def addPredictions(predictions, ticker):
  try:
    # read connection parameters
    params = db.config()

    connection = psycopg2.connect(**params)
    cursor = connection.cursor()

    for p in predictions:
      table_name = ticker.lower()
      if '-usd' in table_name:
        table_name = "pred_crypt_" + table_name[:-4]
      else:
        table_name = "pred_stock_" + table_name
      query = """ 
      INSERT INTO """ + table_name + """ 
      (week, price, prediction, posts)
      VALUES
      (%s,%s,%s,%s)
      """

      cursor.execute(query, (p[0], p[1], p[2], p[3]))
      connection.commit()

  except (Exception, psycopg2.Error) as error:
    print("Failed to select a record from the table: ", error)

  finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()

def createPredictionTables():
  try:
    table_names = []

    for ticker in prices.topstocks:
      ticker = ticker.lower()
      ticker = ticker.replace('-', '_')
      table_names.append('pred_stock_' + ticker)

    for ticker in prices.topcryptos:
      ticker = str(ticker[:-4])
      ticker = ticker.lower()
      ticker = ticker.replace('-', '_')
      table_names.append('pred_crypt_' + ticker)

    params = db.config()

    connection = psycopg2.connect(**params)
    cursor = connection.cursor()

    for name in table_names:
      create_stock_table_query = "CREATE TABLE " + str(name) + " (week BIGINT PRIMARY KEY, price FLOAT DEFAULT 0.000, prediction FLOAT DEFAULT 0.000, posts BIGINT DEFAULT 0);"
      cursor.execute(create_stock_table_query)
      connection.commit()

  except (Exception, psycopg2.Error) as error:
    print("Failed to create a stock table: ", error)

  finally:
    if connection:
      cursor.close()
      connection.close()

def savePlot(preds, ticker):
  weeks = [x[0] for x in preds]
  act_prices = [x[1] for x in preds]
  pred_prices = [x[2] for x in preds]

  plt.plot(weeks, act_prices)
  plt.plot(weeks, pred_prices)
  plt.legend(["factual price", "predicted price"], loc="upper right")
  plt.title(ticker)
  plt.savefig("../../Plots/" + ticker)
  plt.clf()

def saveCSV(preds, ticker):
  df = pd.DataFrame(preds, columns=['week', 'price', 'prediction', 'posts'])
  df.to_csv("../../Predictions/" + ticker + ".csv", index=False)

In [66]:
(pd.to_datetime('1.1.2019', dayfirst=True) + pd.DateOffset(days = (261) * 7)).date()

datetime.date(2024, 1, 2)

In [68]:
from IPython.display import clear_output
import concurrent.futures
import prices
import math

def prepare_for_analysis(curTicker):
  preds = retrievePredictions(curTicker)
  if preds == -1:
    return
  relevant_preds = preds[260:]
  initial_market_price = -1
  initial_predic_price = -1
  for i in range(len(relevant_preds)):
    if math.isnan(relevant_preds[i][1]) or math.isnan(relevant_preds[i][2]):
      continue
    if relevant_preds[i][1] == -1 or relevant_preds[i][2] == 0.0:
      continue
    initial_market_price = relevant_preds[i][1]
    initial_predic_price = relevant_preds[i][2]
    break

  abs_diff = [abs(relevant_preds[i][1] - relevant_preds[i][2]) for i in range(len(relevant_preds))]
  market_change = [(relevant_preds[i][1] - initial_market_price) / initial_market_price if initial_market_price != 0 else 0 for i in range(len(relevant_preds))]
  predic_change = [(relevant_preds[i][2] - initial_predic_price) / initial_predic_price if initial_predic_price != 0 else 0 for i in range(len(relevant_preds))]
  market_weekly_change = [0] + [(relevant_preds[i][1] - relevant_preds[i-1][1]) / relevant_preds[i-1][1] if relevant_preds[i-1][1] != 0 else 0 for i in range(1, len(relevant_preds))]
  predic_weekly_change = [0] + [(relevant_preds[i][2] - relevant_preds[i-1][2]) / relevant_preds[i-1][2] if relevant_preds[i-1][2] != 0 else 0 for i in range(1, len(relevant_preds))]
  abs_pred_err_general = [abs(market_change[i]-predic_change[i]) for i in range(len(market_change))]
  abs_pred_err_weekly = [abs(market_weekly_change[i]-predic_weekly_change[i]) for i in range(len(market_weekly_change))]

  finalPredTable = []
  for i in range(len(relevant_preds)):
    entry = []
    for r in relevant_preds[i]:
      entry.append(r)
    entry.append(abs_diff[i])
    entry.append(market_change[i])
    entry.append(predic_change[i])
    entry.append(abs_pred_err_general[i])
    entry.append(market_weekly_change[i])
    entry.append(predic_weekly_change[i])
    entry.append(abs_pred_err_weekly[i])
    finalPredTable.append(tuple(entry))

  df = pd.DataFrame(finalPredTable, columns=['week', 'price', 'prediction', 'posts', 'absolute_diff', 'market_change', 'prediction_change', 'error_general', 'market_weekly_change', 'prediction_weekly_change', 'error_weekly'])
  df.to_csv("../../Analysis/" + curTicker + ".csv", index=False)

  accuracyScoreGeneral = [0, 0]
  accuracyScoreWeekly = [0, 0]
  averageGeneralAbsoluteError = [0, 0]
  averageWeeklyAbsoluteError = [0, 0]
  for i in range(len(finalPredTable)):
    if relevant_preds[i][1] == -1 or relevant_preds[i][2] == 0.0 or relevant_preds[i][3] == 0:
      continue
    if  market_change[i] != 0 and predic_change != 0:
      if market_change[i] * predic_change[i] > 0:
        accuracyScoreGeneral[0] += 1
        averageGeneralAbsoluteError[0] += abs_pred_err_general[i]
        averageGeneralAbsoluteError[1] += 1
      accuracyScoreGeneral[1] += 1
    if  market_weekly_change[i] != 0 and predic_weekly_change != 0:
      if market_weekly_change[i] * predic_weekly_change[i] > 0:
        accuracyScoreWeekly[0] += 1
        averageWeeklyAbsoluteError[0] += abs_pred_err_weekly[i]
        averageWeeklyAbsoluteError[1] += 1
      accuracyScoreWeekly[1] += 1
      
  averageGeneralAbsoluteError[0] = averageGeneralAbsoluteError[0] / averageGeneralAbsoluteError[1] if averageGeneralAbsoluteError[1] != 0 else 0
  averageWeeklyAbsoluteError[0] = averageWeeklyAbsoluteError[0] / averageWeeklyAbsoluteError[1] if averageWeeklyAbsoluteError[1] != 0 else 0

  print(curTicker, accuracyScoreGeneral[0], accuracyScoreGeneral[1], accuracyScoreWeekly[0], accuracyScoreWeekly[1], averageGeneralAbsoluteError[0], averageWeeklyAbsoluteError[0])

In [69]:
topTickers = prices.topstocks | prices.topcryptos

def worker(tticker):
  prepare_for_analysis(tticker)

pool = concurrent.futures.ThreadPoolExecutor(max_workers=100)

counter = 0
for tticker in topTickers:
  counter += 1
  pool.submit(worker(tticker))
 
pool.shutdown(wait=True)
print("Main thread continuing to run")

WMB 13 24 10 24 0.07756486750667374 0.027895202728181533
CDNS 22 24 15 24 0.037623608736395586 0.021609010598670764
DIS 3 24 11 24 0.010272140561330507 0.015545955019678143
PGR 24 24 13 24 0.1655951339444125 0.015024689837461015
REG 22 24 11 24 0.04963121400165028 0.01740660328830946
GLW 0 0 0 0 0 0
MOS 18 24 11 24 0.10900980683961893 0.023804943177341224
AIZ 16 24 10 24 0.04730208586628814 0.015540419097168362
BR 9 24 13 24 0.14923393957305375 0.017334378673552704
EMN 10 24 10 24 0.05706675694716253 0.018949634837679953
KLAC 0 0 0 0 0 0
UNH 0 0 0 0 0 0
WAT 14 24 11 24 0.03534314790472916 0.029428117354210317
DVA 1 24 11 24 0.013551434099833294 0.009974255614285667
SLB 0 0 0 0 0 0
CHTR 9 24 13 24 0.1441408360512019 0.0203540351624182
AOS 0 0 0 0 0 0
ROL 10 23 11 23 0.0335570944407531 0.014593672493027275
HD 13 24 12 24 0.028002717644619528 0.016078318210563543
ADA-USD 21 24 13 24 0.0838469400609893 0.07571148105592614
LYB 21 24 14 24 0.032807499684620885 0.013245572758952533
NKE 0 0 0 

In [ ]:
import pandas as pd

df = pd.read_csv('./Comparisons.csv', sep = " ")
df['Performance_General'] = df['Correct_General'] / df['Total_General']
df['Performance_Weekly'] = df['Correct_Weekly'] / df['Total_Weekly']

df = df.sort_values(by='Performance_General', ascending=False)
df.to_csv("ComparisonsSortedByGeneralPerformance.csv", index=False)
df = df.sort_values(by='Performance_Weekly', ascending=False)
df.to_csv("ComparisonsSortedByWeeklyPerformance.csv", index=False)
df = df.sort_values(by='AverageGeneralError', ascending=False)
df.to_csv("ComparisonsSortedByAverageGeneralError.csv", index=False)
df = df.sort_values(by='AverageWeeklyError', ascending=False)
df.to_csv("ComparisonsSortedByAverageWeeklyError.csv", index=False)

In [ ]:
from IPython.display import clear_output
import concurrent.futures
import prices

topTickers = prices.topcryptos | prices.topcryptos

def worker(tticker):
  clear_output(wait=False)
  preds = retrievePredictions(tticker)
  if preds == -1:
    return
  # addPredictions(preds, tticker)
  savePlot(preds, tticker)
  saveCSV(preds, tticker)

pool = concurrent.futures.ThreadPoolExecutor(max_workers=1000)

counter = 0
for tticker in topTickers:
  counter += 1
  pool.submit(worker(tticker))
  clear_output(wait=False)
  print(counter)
 
pool.shutdown(wait=True)
print("Main thread continuing to run")

Simulation

In [127]:
def simulate_weekly(tickerName):
  try:
    preds = pd.read_csv("../../Analysis/" +  tickerName + ".csv")
  except:
    return
  wallet = {}
  cash = 0.0
  wallet[tickerName] = 10
  cash -= preds.iloc[0].tolist()[1] * wallet[tickerName]
  weeks = 0
  for i in range(len(preds)):
    curRow = preds.iloc[i].tolist()
    nextRow = preds.iloc[i+1].tolist()
    if nextRow[1] == -1.0 or nextRow[2] == 0.0 or nextRow[3] == 0:
      cash += wallet[tickerName] * curRow[1]
      weeks = i + 1
      break
    # prediction_weekly_change
    if nextRow[9] > 0:
      cash -= curRow[1]
      wallet[tickerName] += 1
    else:
      if wallet[tickerName] == 0:
        weeks = i + 1
        continue
      cash += curRow[1]
      wallet[tickerName] -= 1
  return cash, weeks

def simulate_general(tickerName):
  try:
    preds = pd.read_csv("../../Analysis/" +  tickerName + ".csv")
  except:
    return
  initialPrice = preds.iloc[0].tolist()[1]
  wallet = {}
  wallet[tickerName] = 10
  cash = 0.0
  cash -= preds.iloc[0].tolist()[1] * wallet[tickerName]
  for i in range(len(preds)):
    curRow = preds.iloc[i].tolist()
    nextRow = preds.iloc[i+1].tolist()
    if nextRow[1] == -1.0 or nextRow[2] == 0.0 or nextRow[3] == 0:
      cash += wallet[tickerName] * curRow[1]
      break
    # prediction_weekly_change
    predictedPrice = (nextRow[6] * initialPrice) + initialPrice
    if predictedPrice > curRow[1]:
      cash -= curRow[1]
      wallet[tickerName] += 1
    else:
      if wallet[tickerName] == 0:
        continue
      cash += curRow[1]
      wallet[tickerName] -= 1
  return cash

def simulate_worker(tickerName):
  if simulate_weekly(tickerName) == None or simulate_general(tickerName) == None:
    return
  profit_weekly, weeks = simulate_weekly(tickerName)
  profit_general = simulate_general(tickerName)
  if profit_weekly == None or profit_general == None or weeks == 0:
    return
  if profit_weekly == 0 or profit_general == 0:
    return
  print(tickerName, profit_weekly, profit_general, weeks)

In [129]:
pool = concurrent.futures.ThreadPoolExecutor(max_workers=100)
for tticker in topTickers:
  pool.submit(simulate_worker(tticker))
pool.shutdown(wait=True)
print("Main thread continuing to run")

WMB 92.19348907470703 25 18.213706970214844 25
CDNS 288.6199645996094 25 553.7999572753906 25
DIS 120.35005187988281 25 173.2100372314452 25
PGR 649.4575805664072 25 254.53408813476562 25
REG -53.79465866088856 25 -41.46285629272438 25
MOS -103.48493003845215 25 -133.44900703430176 25
AIZ 21.522003173828125 25 68.47184753417969 25
BR -100.19929504394531 25 -121.71293640136719 25
EMN 163.1155548095703 25 261.5596160888672 25
WAT -243.26998901367188 25 138.04998779296875 25
DVA 167.54994964599587 25 67.25997161865212 25
CHTR -1184.6403198242188 25 -1559.6404418945312 25
ROL 39.540260314941406 25 24.98566436767578 25
HD -296.989013671875 25 -349.1556396484375 25
ADA-USD -1.5426177680492401 25 -0.04116579890251165 25
LYB -1.3229904174809235 25 37.6992568969722 25
CHD 191.41422271728516 25 79.264892578125 25
SYF 35.535438537597656 25 6.022838592529304 25
HPE 56.89995765686035 25 88.23198699951172 25
HLT 249.5266876220703 25 131.15802001953125 25
BLDR -248.2400360107422 25 200.2799530029297 

In [111]:
selectedTickers = ["AAPL"]
for tickerName in selectedTickers:
  

-2120.8438415527344 11
-2306.2472534179688 12
-2491.1513061523438 13
-2674.5472869873047 14
-2869.478546142578 15
-3057.2789001464844 16
-3246.337661743164 17
-3431.377655029297 18
-3612.9376525878906 19
-3795.567657470703 20
-3965.6876525878906 21
-4138.91764831543 22
-4314.997650146484 23
-4484.707656860352 24
-4653.547653198242 25
-4823.2176513671875 26
-4992.59765625 27
-5159.497650146484 28
-5329.827651977539 29
-5512.227645874023 30
-5699.657638549805 31
-5892.00764465332 32
-6081.997650146484 33
-6276.34765625 34
766.7521362304697
